# extract site data from DeepMIP models

This notebook finds and displays all available DeepMIP model data at a user-defined location. You can either run the notebook along the code in the Jupyter Notebook environment or click on the `Voilà` button at the top for a dashboard view.

**Minimum input:** The variable name you want to process and the present-day latitude (between -90.0 and 90.0) and longitude (between -180.0 and 180.0) of your site. The paleolocation is derived internally from the Herold et al. (2014) paleogeography to be consistent with the model land-sea mask.

**Optional input:** You can specify a label for the site name and a range of reconstructed proxy values to be displayed along the model results.

**Output:**
The following mean metrics are calculated from climatological monthly mean data: *annual*, *monthly minimum*, *monthly maximum*, *December to February*, *March to May*, *June to August* and *September to November*. 
- Figure 1: Early Eocene (55Ma) paleogeographic map with the rotated site in equirectangular and orthographic projection. The coastlines and hollow circles on the maps indicate the present-day geography for reference.
- Figure 2: Boxplots of simulated values at the paleolocation grouped by experiment (i.e. increasing CO2). Pre-industrial values (at the modern location) are shown to indicate intermodel variability. Boxplots describe distribution of data abesd on: minimum, first quartile, median, third quartile, maximum and outliers.
- Table 3: Overview of calculated metrics for each available model simulation.


Instructions on how to download and run the python code locally can be found at:
https://github.com/sebsteinig/DeepMIP_model_database_notebooks.


In [ ]:
work_dir       = './'
data_dir       = work_dir + 'DeepMIP-Eocene/User_Model_Database_v1.0/'

In [ ]:
# load packages
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import cartopy.crs as ccrs
import cmocean
import geoviews as gv
import holoviews as hv
import geoviews.feature as gf
import panel as pn

from pathlib import Path

from cartopy.util import add_cyclic_point
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy import crs
from geoviews import opts

# dictionaries containing info about DeepMIP models and variables
from deepmipModelDict import deepmipModelDict
from deepmipVariableDict import deepmipVariableDict

gv.extension('bokeh')

In [ ]:
# open Herold et al. (2014) rotation file
rotationFile = xr.open_dataset(work_dir + 'LatLon_PD_55Ma_Herold2014.nc') 

# open Herold et al. (2014) paleogeography
geography = xr.open_dataset(work_dir + 'herold_etal_eocene_topo_1x1.nc').topo
lons = xr.open_dataset(work_dir + 'herold_etal_eocene_topo_1x1.nc').lon
lats = xr.open_dataset(work_dir + 'herold_etal_eocene_topo_1x1.nc').lat

# add cyclic longitude for plotting
geography, lonsc = add_cyclic_point(geography, lons)
levels = np.linspace(-5400,5400, num=21)

df = pd.DataFrame()

expts = ['piControl', 'deepmip_sens_1xCO2', 'deepmip_sens_2xCO2', 'deepmip_stand_3xCO2', 'deepmip_sens_4xCO2', 'deepmip_stand_6xCO2', 'deepmip_sens_9xCO2']
exptLabels = ['piControl', 'DeepMIP_1x', 'DeepMIP_2x', 'DeepMIP_3x', 'DeepMIP_4x', 'DeepMIP_6x', 'DeepMIP_9x']

    
# find paleoposition for DeepMIP model geography
def rotateSite(modernLat, modernLon):

    # 1. coarse approximation: look up paleolocation for modern coordinates in rotation file
    paleoLat = rotationFile.LAT.sel(latitude=modernLat, longitude=modernLon, method='nearest').values
    paleoLon = rotationFile.LON.sel(latitude=modernLat, longitude=modernLon, method='nearest').values
    # 2. fine approximation: add delta between modern selected and rotation grid coordinates back to paleolocation
    deltaLat = modernLat - rotationFile.latitude.sel(latitude=modernLat, method='nearest').values
    deltaLon = modernLon - rotationFile.longitude.sel(longitude=modernLon, method='nearest').values
    paleoLat += deltaLat
    paleoLon += deltaLon
    
    return paleoLat, paleoLon


# plot paleoposition on paleogeographic map
def plotSite():    
    
    modernPoints = dict(
        Longitude = [longitude.value],
        Latitude  = [latitude.value])
    
    paleoLat, paleoLon = rotateSite(latitude.value, longitude.value)
   
    rotatedPoints = dict(
        Longitude = [paleoLon],
        Latitude  = [paleoLat])
    
    userProjection = getattr(crs, mapProjection.value)

    filledPoints = gv.Points(rotatedPoints, crs=crs.PlateCarree()).opts(size=12, color='red', line_color='black')
    contours = gv.FilledContours((lonsc, lats, geography), crs=crs.PlateCarree())     
    overlay = contours * filledPoints
    
    if checkboxCoastline.value:
        hollowPoints = gv.Points(modernPoints, crs=crs.PlateCarree()).opts(size=12, fill_alpha=0, line_color='red')
        coastline = gf.coastline(line_width=2, line_color='grey')       
        overlay *= hollowPoints * coastline
        
    if siteLabel.value != 'untitled':
        labels = gv.Labels((paleoLon + offsetLongitude.value, paleoLat + offsetLatitude.value, siteLabel.value), crs=crs.PlateCarree()).opts(text_color='white', text_align='left',text_font_size='12pt', text_font_style = 'bold')
        overlay *= labels
        
    if mapProjection.value == 'Orthographic':  
        return (overlay).opts(
                    opts.FilledContours(levels=levels, line_alpha=0.0, cmap='cmo.topo', color_levels=20,
                                        colorbar=True, data_aspect=1, width=940, height=400, projection=userProjection(central_longitude=centerLongitude.value, central_latitude=centerLatitude.value),
                                        title='55Ma paleogeography (Herold et al., 2014)' ))     
    else:
        return (overlay).opts(
                    opts.FilledContours(levels=levels, line_alpha=0.0, cmap='cmo.topo', color_levels=20,
                                        colorbar=True, data_aspect=1, width=940, height=400, projection=userProjection(central_longitude=centerLongitude.value),
                                        title='55Ma paleogeography (Herold et al., 2014)' ))
    
# load model data at paleoposition 
def loadData( modernLat, modernLon, userVariable):
        
    # allocate empty list to store results for all models
    siteDataList = []
    
    # look up DeepMIP variable name
    for variable in deepmipVariableDict.keys():
        if deepmipVariableDict[variable]['longname'] == userVariable:
            deepmipVariable = variable
            varUnit = deepmipVariableDict[variable]['unit']

    # loop over all models and experiments
    for expCount, exp in enumerate(expts):
        for modelCount, model in enumerate(deepmipModelDict.keys()):     

            # construct filename following the DeepMIP convention
            modelFile = data_dir + deepmipModelDict[model]['group'] + '/' + model + '/' + exp + '/' + deepmipModelDict[model]['versn'] + \
                        '/' + model + '-' + exp + '-' + deepmipVariable + '-' + deepmipModelDict[model]['versn'] + '.mean.nc'

            # load data if file for model/experiment combination exists
            if Path(modelFile).exists():
                modelDataset = xr.open_dataset(modelFile, decode_times=False)

                # get coordinate names
                for coord in modelDataset.coords:
                    if coord in ['lat', 'latitude']:
                        latName = coord
                    elif coord in ['lon', 'longitude']:
                        lonName = coord

                if exp == 'piControl':
                    lookupLat = modernLat
                    lookupLonOrig = modernLon
                else:           
                    lookupLat, lookupLonOrig = rotateSite(modernLat, modernLon)
        
                # check for minimum model longitude
                minModelLon = np.amin(modelDataset.coords[lonName].values)
                if minModelLon >= 0.0 and lookupLonOrig < 0.0:
                    # convert lookupLon from [-180:180] to [0:360]
                    lookupLon = lookupLonOrig + 360.0 
                else:
                    lookupLon = lookupLonOrig

                varData = getattr(modelDataset, deepmipVariable)
                if deepmipVariable == 'tas':
                    # convert from Kelvin to Celsius
                    siteData = varData.sel(**{latName: lookupLat}, **{lonName: lookupLon}, method='nearest').values - 273.15
                elif deepmipVariable == 'pr':
                    # convert from kg m-2 s-1 to mm/day
                    siteData = varData.sel(**{latName: lookupLat}, **{lonName: lookupLon}, method='nearest').values * 86400.
                else:
                    siteData = varData.sel(**{latName: lookupLat}, **{lonName: lookupLon}, method='nearest').values

                # store results for individual metrics in a dictionary
                siteDataList.append(dict(model = model, 
                                         modelShort = deepmipModelDict[model]['abbrv'],
                                         experiment = exptLabels[expCount], 
                                         site = siteLabel.value,
                                         lat = np.round(lookupLat, 2),
                                         lon = np.round(lookupLonOrig,2),
                                         var = deepmipVariable,
                                         unit = varUnit,
                                         annualMean = np.mean(siteData), 
                                         monthlyMin = np.min(siteData), 
                                         monthlyMax = np.max(siteData), 
                                         DJF = np.mean(siteData[[11,0,1]]), 
                                         MAM = np.mean(siteData[[2,3,4]]), 
                                         JJA = np.mean(siteData[[5,6,7]]), 
                                         SON = np.mean(siteData[[8,9,10]]),
                                         Jan = siteData[0],
                                         Feb = siteData[1],
                                         Mar = siteData[2],
                                         Apr = siteData[3],
                                         May = siteData[4],
                                         Jun = siteData[5],
                                         Jul = siteData[6],
                                         Aug = siteData[7],
                                         Sep = siteData[8],
                                         Oct = siteData[9],
                                         Nov = siteData[10],
                                         Dec = siteData[11] ))

    # convert dictionary to Pandas dataframe for easier handling and plotting  
    df = pd.DataFrame(siteDataList)
    
    # calculate ensemble mean for each experiment
    for exp in exptLabels:  
        df.loc[len(df)] = df.loc[df['experiment'] == exp].mean()
        # set ensemble mean labels
        df.loc[len(df)-1,'model'] = 'ensemble_mean'
        df.loc[len(df)-1,'modelShort'] = 'ensemble_mean'
        df.loc[len(df)-1,'experiment'] = exp
        df.loc[len(df)-1,'site'] = siteLabel.value
        df.loc[len(df)-1,'var'] = deepmipVariable
        df.loc[len(df)-1,'unit'] = varUnit
        
    return(df)

def getData(event):
    df = loadData( latitude.value, longitude.value, variable.value)
    dfWidget.value = df
    updateMap.value = not updateMap.value
    return df



In [ ]:
import panel.widgets as pnw
from io import StringIO
from bokeh.models.widgets.tables import NumberFormatter

bootstrap = pn.template.BootstrapTemplate(title='DeepMIP database tools', logo='deepmip_logo.png')
#pn.config.sizing_mode = 'stretch_width'

css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
  overflow: scroll;
}
'''

pn.extension(raw_css=[css])
pn.param.ParamMethod.loading_indicator = True

# define buttons for sidebar navigation
siteNavButton = pn.widgets.Button(name='Site Data', button_type='default', disabled=False)
zmNavButton = pn.widgets.Button(name='Zonal Mean', button_type='default', disabled=True)
gmNavButton = pn.widgets.Button(name='Global Mean', button_type='default', disabled=True)
mapNavButton = pn.widgets.Button(name='Surface Maps', button_type='default', disabled=True)
aboutNavButton = pn.widgets.Button(name='About', button_type='default', disabled=True)

bootstrap.sidebar.append(pn.pane.Markdown("## Navigation \nSelect type of analysis. You can close the sidebar at the top. "))
bootstrap.sidebar.append(siteNavButton)
bootstrap.sidebar.append(zmNavButton)
bootstrap.sidebar.append(gmNavButton)
bootstrap.sidebar.append(mapNavButton)
bootstrap.sidebar.append(aboutNavButton)

###################################################################################################################
# TopLeft: description and main input controls
###################################################################################################################

textIntro1 = pn.pane.Markdown("""
\n### Get model data for single site 
\nThis dashboard finds and displays all available DeepMIP model data closest to a user-defined location.

**Minimum input:** The variable name you want to process and the present-day latitude (between -90.0 and 90.0) and longitude (between -180.0 and 180.0) of your site. The paleolocation is derived internally from the Herold et al. (2014) paleogeography to be consistent with the model land-sea mask.

""", background='white', height=175,style={'font-size': '18px'})

textIntro2 = pn.pane.Markdown("""
**Optional input:** You can specify a label for the site name and a range of reconstructed proxy values to be displayed along the model results.

**Output:**
The following mean metrics are calculated from climatological monthly mean data: *annual*, *monthly minimum*, *monthly maximum*, *December to February*, *March to May*, *June to August* and *September to November*. 
\nInstructions on how to download and run the underlying python code locally can be found at:
https://github.com/sebsteinig/DeepMIP_model_database_notebooks
""", background='white', height=255,style={'font-size': '18px'})

variable = pn.widgets.Select(name='variable', value='near-surface air temperature', options=['near-surface air temperature', 'precipitation'],width=215)
latitude  = pn.widgets.FloatInput(name='modern latitude', placeholder='-90.0 to 90.0', value=-43.1, start=-90., end=90., width=130)
longitude = pn.widgets.FloatInput(name='modern longitude', placeholder='-180.0 to 180.0', value=172.7, start=-180., end=180., width=130)
siteLabel = pn.widgets.TextInput(name='site label', placeholder='optional', value='Mid-Waipara River', width=175)

runButton = pn.widgets.Button(name='Get Data', button_type='primary',align='end', width=180)

mandatoryInput = pn.Row(variable, latitude, longitude, siteLabel)

#inputPane = pn.Column(textIntro, pn.Row(pn.Column(variable, latitude, longitude),pn.Column(proxyMean, proxyMin, proxyMax), pn.Column(siteLabel, centerLongitude)), runButton, css_classes=['panel-widget-box'], sizing_mode='fixed', width=935, height=530, margin=3) 
inputPane = pn.Column(textIntro1, pn.Row(mandatoryInput, runButton, css_classes=['panel-widget-box'], sizing_mode='fixed', width=935, height=60, margin=3), textIntro2 )

###################################################################################################################
# TopRight: paleogeography plot
###################################################################################################################

mapProjection = pn.widgets.Select(name='projection', value='PlateCarree', options=['PlateCarree', 'Orthographic', 'Robinson', 'Mercator'],width=138)
centerLongitude = pn.widgets.FloatInput(name='center longitude', placeholder='-180.0 to 180.0', value=0.0, start=-180., end=180.,width=138)
centerLatitude = pn.widgets.FloatInput(name='center latitude', placeholder='-90.0 to 90.0', disabled=True, value=-45.0, start=-90., end=90.,width=138)
offsetLongitude = pn.widgets.FloatInput(name='label longitude offset', placeholder='in degrees', value=0.0,width=138)
offsetLatitude = pn.widgets.FloatInput(name='label latitude offset', placeholder='in degrees', value=7.0,width=138)
checkboxCoastline = pn.widgets.Checkbox(name='modern coastline', value=True ,align='center',width=138)
updateMap = pn.widgets.Toggle(value=False)

def toggleCenterLat(projection):
    if projection.new == 'Orthographic':
        centerLatitude.disabled = False
    else:
        centerLatitude.disabled = True
    
mapProjection.param.watch(toggleCenterLat, 'value')

mapInput = pn.Row(mapProjection, centerLongitude, centerLatitude, offsetLongitude, offsetLatitude, checkboxCoastline)

@pn.depends(mapProjection, centerLongitude, centerLatitude, offsetLongitude, offsetLatitude, checkboxCoastline, updateMap)
def mapPlot(mapProjection, centerLongitude, centerLatitude, offsetLongitude, offsetLatitude, checkboxCoastline, updateMap):  

    figure = plotSite()
    
    return figure

mapLegend = pn.pane.Markdown("""**Figure 1:** Early Eocene (55Ma) paleogeographic map and paleolocation for the selected site. The coastline and hollow circle indicate the present-day geography and site location for reference. Use the toolbar on the right to pan, zoom and download the figure.""")
mapPane = pn.Column(mapInput, mapPlot, mapLegend, sizing_mode='fixed', width=955, height=530, css_classes=['panel-widget-box'], margin=3)

###################################################################################################################
# BottomLeft: dataframe table
###################################################################################################################
df = loadData(latitude.value, longitude.value, variable.value)
bokeh_formatters = {'lat': NumberFormatter(format='0.0'),'lon': NumberFormatter(format='0.0'),'annualMean': NumberFormatter(format='0.0'),'monthlyMin': NumberFormatter(format='0.0'),'monthlyMax': NumberFormatter(format='0.0'),'DJF': NumberFormatter(format='0.0'),'MAM': NumberFormatter(format='0.0'),'JJA': NumberFormatter(format='0.0'),'SON': NumberFormatter(format='0.0'),'Jan': NumberFormatter(format='0.0'),'Feb': NumberFormatter(format='0.0'),'Mar': NumberFormatter(format='0.0'),'Apr': NumberFormatter(format='0.0'),'May': NumberFormatter(format='0.0'),'Jun': NumberFormatter(format='0.0'),'Jul': NumberFormatter(format='0.0'),'Aug': NumberFormatter(format='0.0'),'Sep': NumberFormatter(format='0.0'),'Oct': NumberFormatter(format='0.0'),'Nov': NumberFormatter(format='0.0'),'Dec': NumberFormatter(format='0.0')}
dfWidget = pn.widgets.Tabulator(df, formatters=bokeh_formatters)

expSelect = pn.widgets.CheckButtonGroup(name='experiments', value=['piControl', 'DeepMIP_1x', 'DeepMIP_2x', 'DeepMIP_3x', 'DeepMIP_4x', 'DeepMIP_6x', 'DeepMIP_9x'], options=['piControl', 'DeepMIP_1x', 'DeepMIP_2x', 'DeepMIP_3x', 'DeepMIP_4x', 'DeepMIP_6x', 'DeepMIP_9x'], button_type='default')
modelSelect = pn.widgets.CheckButtonGroup(name='experiments', value=['CESM', 'COSMOS', 'GFDL', 'HadCM3', 'HadCM3L', 'INM', 'IPSL','MIROC', 'ensemble_mean'], options=['CESM', 'COSMOS', 'GFDL', 'HadCM3', 'HadCM3L', 'INM', 'IPSL','MIROC', 'ensemble_mean'], button_type='default')
statSelect = pn.widgets.CheckButtonGroup(name='statistics', value=['annualMean', 'monthlyMin', 'monthlyMax', 'DJF', 'MAM', 'JJA', 'SON','all months'], options=['annualMean', 'monthlyMin', 'monthlyMax', 'DJF', 'MAM', 'JJA', 'SON','all months'], button_type='default')

@pn.depends(expSelect, modelSelect, statSelect, dfWidget)
def filtered_df(expSelect, modelSelect, statSelect, dfWidget):
    df = dfWidget
    filteredDF = df[df['experiment'].isin(expSelect) & df['modelShort'].isin(modelSelect)]
    deselectedStatistics = list(set(['annualMean', 'monthlyMin', 'monthlyMax', 'DJF', 'MAM', 'JJA', 'SON','all months']) - set(statSelect))
    hiddenColumns= ['modelShort'] + deselectedStatistics
    if 'all months' in hiddenColumns:
        hiddenColumns.remove('all months')
        hiddenColumns = hiddenColumns + ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return pn.widgets.Tabulator(filteredDF,layout='fit_data', show_index=False, hidden_columns=hiddenColumns, disabled=True, width=915, formatters=bokeh_formatters)

@pn.depends(expSelect, modelSelect, statSelect, dfWidget)
def filtered_file(expSelect, modelSelect, statSelect, dfWidget):
    df = filtered_df(expSelect, modelSelect, statSelect, dfWidget)
    sio = StringIO()
    deselectedStatistics = list(set(['annualMean', 'monthlyMin', 'monthlyMax', 'DJF', 'MAM', 'JJA', 'SON','all months']) - set(statSelect))
    hiddenColumns= ['modelShort'] + deselectedStatistics
    if 'all months' in hiddenColumns:
        hiddenColumns.remove('all months')
        hiddenColumns = hiddenColumns + ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    df.value.drop(columns=hiddenColumns).to_csv(sio, index=False, float_format="%.2f")
    sio.seek(0)
    return sio

download = pn.widgets.FileDownload(
    callback=filtered_file, filename='deepmipSiteData.csv', button_type='primary'
)

tableCaption = pn.pane.Markdown("""**Table 1:** Overview of extracted data and calculated metrics for each available model simulation. Use the boxes at the top to filter the results by experiment, model and statistic and download the displayed table as a CSV file.""", height=35)
#tablePane = pn.Column(tableCaption, pn.Row(filename, button), dfWidget, sizing_mode='fixed', width=950, css_classes=['panel-widget-box'], margin=5)
tablePane = pn.Column(tableCaption, expSelect, modelSelect, pn.Row(pn.Column(statSelect, width=700, align='end'), pn.Column(download, width=235, align='end')), filtered_df, sizing_mode='fixed', width=935, height=530, css_classes=['panel-widget-box'], margin=3)

###################################################################################################################
# BottomRight: whiskers plot
###################################################################################################################

boxStatistic = pn.widgets.Select(name='boxplot metric', value='annualMean', options=['annualMean', 'monthlyMin', 'monthlyMax', 'DJF', 'MAM', 'JJA', 'SON','Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
checkboxProxy = pn.widgets.Checkbox(name='compare to proxy', value=True ,align='center')
proxyMean  = pn.widgets.FloatInput(name='proxy mean', placeholder='optional', value=20.0)
proxyMin   = pn.widgets.FloatInput(name='proxy minimum', placeholder='optional', value=18.9)
proxyMax   = pn.widgets.FloatInput(name='proxy maximum', placeholder='optional', value=21.5)
proxyLabel = pn.widgets.TextInput(name='proxy label', placeholder='optional', value='EECO MBT\'-CBT')

def toggleInput(checkbox):
    if checkbox.new == True:
        proxyMean.disabled = proxyMin.disabled = proxyMax.disabled = proxyLabel.disabled = False
    else:
        proxyMean.disabled = proxyMin.disabled = proxyMax.disabled = proxyLabel.disabled = True
    
checkboxProxy.param.watch(toggleInput, 'value')

@pn.depends(expSelect, modelSelect, statSelect, dfWidget, boxStatistic, checkboxProxy, proxyMin, proxyMax, proxyMean, proxyLabel)
# boxplot of model data
def boxPlot(expSelect, modelSelect, statSelect, dfWidget, boxStatistic, checkboxProxy, proxyMin, proxyMax, proxyMean, proxyLabel):  
    
#    df = dfWidget
    df = filtered_df(expSelect, modelSelect, statSelect, dfWidget)
    dfPlot = df.value
    dfPlot = dfPlot[(dfPlot.model != 'ensemble_mean')]
    dfPlot = dfPlot.loc[dfPlot['site'] == siteLabel.value]

    dfEocene = dfPlot.loc[dfPlot['experiment'] != 'piControl']
    
    box = hv.BoxWhisker(dfPlot,
                         kdims=['experiment'],
                         vdims=[boxStatistic]
                        ).opts(
                        opts.BoxWhisker(box_color='white', width=940, height=400, show_legend=False, whisker_color='black',box_fill_color='#63c5da', 
                                        title='"'+ siteLabel.value + '" DeepMIP model results (pLAT = ' + str(np.round(dfEocene['lat'].iloc[0])) + ' / pLON = ' + str(np.round(dfEocene['lon'].iloc[0])) +')',
                                        ylabel=boxStatistic + ' ' + variable.value + ' [' + str(dfEocene['unit'].iloc[0]) + ']' )) 

    scatter = hv.Scatter(dfPlot,
                     kdims=['experiment'],
                     vdims=[boxStatistic, 'model']
                    ).groupby(
                        'model'
                    ).overlay(
                    ).opts(
                        opts.Scatter(jitter=0.2, width=940, height=400, show_legend=True, legend_position='right', legend_offset=(0, 119), size=12, tools=['hover', 'wheel_zoom'], line_color='black', fontsize={'legend': 10})) 
    

    composition = box * scatter
    
    if checkboxProxy:
        
        if proxyMin != None and proxyMax != None:

            proxyRange  = hv.HSpan(proxyMin, proxyMax).opts(fill_color='#ff9999', line_alpha=0, apply_ranges=True, level='underlay')

            composition *= proxyRange
        
        if proxyMin != None:
            
            minLine  = hv.HLine(proxyMin).opts(color='#ff4d4d', line_width=2, line_dash='dashed', apply_ranges=True, level='underlay')
            
            composition *= minLine

        if proxyMax != None:
            
            maxLine  = hv.HLine(proxyMax).opts(color='#ff4d4d', line_width=2, line_dash='dashed', apply_ranges=True, level='underlay')
            
            composition *= maxLine

        if proxyMean != None:
            
            meanLine  = hv.HLine(proxyMean).opts(color='#ff4d4d', line_width=2, line_dash='solid', apply_ranges=True, level='underlay')
            
            composition *= meanLine

        if proxyLabel != None:
            
            proxyText  = hv.Text(box['experiment'][0], np.nanmin(np.array([proxyMin, proxyMax], dtype=np.float64)), proxyLabel).opts(color='#ff4d4d', text_align='left', text_baseline='top')


            #if proxyMin != None and proxyMax != None:
                
            #    proxyText  = hv.Text(box['experiment'][0], np.mean([proxyMin, proxyMax]), proxyLabel).opts(text_align='left', text_baseline='bottom')
 
            #else:
        


    
            composition *= proxyText
        
    return composition

boxLegend = pn.pane.Markdown(
"""**Figure 2:** Boxplots of simulated values at the paleolocation grouped by experiment showing: minimum, first 
quartile, median, third quartile, maximum and outliers. Individual experiments and models can be (de-)selected at 
the top of Table 1. Use the toolbar on the right to pan, zoom, hover of the data and download the figure.""")

boxPane = pn.Column(pn.Row(boxStatistic, checkboxProxy, proxyMean, proxyMin, proxyMax, proxyLabel, sizing_mode='stretch_width'), boxPlot, boxLegend, sizing_mode='fixed', width=955, height=530, css_classes=['panel-widget-box'], margin=3)

# update data on button click
df = runButton.on_click(getData)

bootstrap.main.append(
    pn.Column(
        pn.Row(inputPane, mapPane),
        pn.Row(tablePane, boxPane),
    )
)

#bootstrap
bootstrap.servable()
#bootstrap.show()

In [ ]:
hv.help(hv.HSpan)